In [1]:
# default_exp autocoder

In [2]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Auto Coder

> Automatically codes text fields such as open-ended survey questions based on lingustic properties such as topic and sentiment.

In [3]:
#hide
from nbdev.showdoc import *

In [1]:
#export
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from causalnlp.analyzers import ZeroShotClassifier

class Autocoder:
    """
    Autocodes text fields
    """
    def __init__(self, verbose=1):
        """
        Instantiates the Autocoder instance.
        """
        self.v = verbose
        self.zsl = ZeroShotClassifier()


    def _format_to_df(self, results, df):
        d = {}
        for e in results:
            if isinstance(e, dict): e = e.items()
            for tup in e:
                label = tup[0]
                prob = tup[1]
                lst = d.get(label, [])
                lst.append(prob)
                d[label] = lst
        new_df = df.join(pd.DataFrame(d, index=df.index))      
        return new_df
    
    def _binarize_df(self, df, colnames, threshold=0.5):
        """
        Binarizes each column in `colnames` based on threshold.
        """
        for col in colnames:
            df[col] = (df[col] >= threshold).astype(int)
        return df
 
    def _check_columns(self, labels, df):
        """check columns"""
        cols = df.columns.values
        for l in labels:
            if l in cols:
                raise ValueError('There is already a column named %s in your DataFrame.' % (l))
        

    def code_sentiment(self, docs, df, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for positive or negative sentiment
        """
        labels = ['negative', 'positive']
        self._check_columns(labels, df)

        results = self.zsl.predict(docs, labels=labels, include_labels=True, multilabel=False,
                              batch_size=batch_size,
                              nli_template="The sentiment of this movie review is {}.")
        df= self._format_to_df(results, df)   
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df
        
    def code_emotion(self, docs, df, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for emotion
        """
        labels = ['joy', 'anger', 'fear', 'sadness']
        self._check_columns(labels, df)

        results = self.zsl.predict(docs, labels=labels, include_labels=True, multilabel=False,
                              batch_size=batch_size,
                              nli_template="The emotion of this text is {}.")
        df= self._format_to_df(results, df)   
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df       
    
    def code_custom_topics(self, docs, df, labels, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for user-specified topics.
        The `label` field is the name of the topic as a string (or a list of them.)
        """
        self._check_columns(labels, df)
           
        results = self.zsl.predict(docs, labels=labels, include_labels=True, batch_size=8)
        df = self._format_to_df(results, df)    
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df
    
    def code_callable(self, docs, df, fn):
        """
        Autocodes text for any user-specified function
        The `fn` parameter must be a Callable and return a dictionary for each
        text in `docs` where the keys are desired column names and values are scores
        or probabilities.
        """
        
        results = self.zsl.predict(docs, labels=labels, include_labels=True, batch_size=8)
        df = self._format_to_df(results, df)    
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df




In [5]:
show_doc(Autocoder.code_sentiment)

<h4 id="Autocoder.code_sentiment" class="doc_header"><code>Autocoder.code_sentiment</code><a href="__main__.py#L41" class="source_link" style="float:right">[source]</a></h4>

> <code>Autocoder.code_sentiment</code>(**`docs`**, **`df`**, **`batch_size`**=*`8`*, **`binarize`**=*`False`*, **`threshold`**=*`0.5`*)

Autocodes text for positive or negative sentiment

Let's prepare a toy dataset:

In [6]:
ac = Autocoder()
reviews = ["I loved this doctor!", "This doctor was absolutely terrible."]
df = pd.DataFrame({
    'gender': ['female', 'male'],
     'review' : reviews,
      })
df.head()

,gender,review
0,female,I loved this doctor!
1,male,This doctor was absolutely terrible.


After autocoding for sentiment, the dataframe now has extra columns:

In [7]:
result_df = ac.code_sentiment(df['review'].values, df)
result_df.head()

,gender,review,negative,positive
0,female,I loved this doctor!,0.005034,0.994966
1,male,This doctor was absolutely terrible.,0.981789,0.018211


In [8]:
assert result_df[result_df['gender']=='female']['negative'].values[0] < 0.1
assert result_df[result_df['gender']=='female']['positive'].values[0] > 0.9
assert result_df[result_df['gender']=='male']['negative'].values[0] > 0.9
assert result_df[result_df['gender']=='male']['positive'].values[0] < 0.1

In [9]:
show_doc(Autocoder.code_custom_topics)

<h4 id="Autocoder.code_custom_topics" class="doc_header"><code>Autocoder.code_custom_topics</code><a href="__main__.py#L65" class="source_link" style="float:right">[source]</a></h4>

> <code>Autocoder.code_custom_topics</code>(**`docs`**, **`df`**, **`labels`**, **`batch_size`**=*`8`*, **`binarize`**=*`False`*, **`threshold`**=*`0.5`*)

Autocodes text for user-specified topics.
The `label` field is the name of the topic as a string (or a list of them.)

Let's prepare a toy dataset:

In [10]:
comments = ["What is your favorite sitcom of all time?", 'I cannot wait to vote!']
df = pd.DataFrame({
    'over_18': ['yes', 'no'],
     'comments' : comments,
      })
df.head()

,over_18,comments
0,yes,What is your favorite sitcom of all time?
1,no,I cannot wait to vote!


After autocoding, the dataframe has a new column for each custom topic:

In [11]:
result_df = ac.code_custom_topics(df['comments'].values, df, labels=['television', 'film', 'politics'])
result_df.head()

,over_18,comments,television,film,politics
0,yes,What is your favorite sitcom of all time?,0.981327,0.012260,0.000157
1,no,I cannot wait to vote!,0.000518,0.004943,0.936988


In [12]:
assert result_df[result_df['over_18']=='yes']['television'].values[0] > 0.9
assert result_df[result_df['over_18']=='yes']['film'].values[0] < 0.1
assert result_df[result_df['over_18']=='yes']['politics'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['television'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['film'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['politics'].values[0] > 0.9

In [13]:
show_doc(Autocoder.code_emotion)

<h4 id="Autocoder.code_emotion" class="doc_header"><code>Autocoder.code_emotion</code><a href="__main__.py#L53" class="source_link" style="float:right">[source]</a></h4>

> <code>Autocoder.code_emotion</code>(**`docs`**, **`df`**, **`batch_size`**=*`8`*, **`binarize`**=*`False`*, **`threshold`**=*`0.5`*)

Autocodes text for emotion

In [14]:
comments = ["I'm nervous about tomorrow.", 'I got a promotion at work!',
            "My best friend was in a car accident.", "I hate it when I'm cut off in traffic."]
df = pd.DataFrame({
    'over_18': ['yes', 'no', 'yes', 'yes'],
     'comments' : comments,
      })
df.head()

,over_18,comments
0,yes,I'm nervous about tomorrow.
1,no,I got a promotion at work!
2,yes,My best friend was in a car accident.
3,yes,I hate it when I'm cut off in traffic.


In [15]:
result_df = ac.code_emotion(df['comments'].values, df, binarize=True)
result_df.head()

,over_18,comments,joy,anger,fear,sadness
0,yes,I'm nervous about tomorrow.,0,0,1,0
1,no,I got a promotion at work!,1,0,0,0
2,yes,My best friend was in a car accident.,0,0,0,1
3,yes,I hate it when I'm cut off in traffic.,0,1,0,0


In [2]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_causalinference.ipynb.
Converted 01_autocoder.ipynb.
Converted 02_analyzers.ipynb.
Converted 99_examples.ipynb.
Converted index.ipynb.
